In [1]:
import os
os.environ['PYDEVD_DISABLE_FILE_VALIDATION'] = '1'
import gc
import dask.dataframe as dd
import datetime
import logging
import numpy as np
import pandas as pd
from dask.distributed import Client, progress

import pyarrow.parquet as pq

# Configurar o logging para visualizar os logs de informação
logging.basicConfig(level=logging.INFO)

# Configurar o cliente Dask
client = Client(n_workers=10, threads_per_worker=1, memory_limit='6.4GB')
logging.info(client)

raw_dataset_path = './BTCUSDT-Trades/'
output_base_path = './BTCUSDT-Trades-compressed/'
timestamp = datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
output_path = f'{output_base_path}'

# Função para extrair o número do nome do arquivo
def extract_number(file_name):
    if file_name.startswith("BTCUSDT-Dataset-part-") and file_name.endswith(".parquet"):
        return int(file_name.split("-")[-1].split(".")[0])  # Extrai o número
    return -1  # Para arquivos que não seguem o padrão (como .DS_Store)

# Função para converter tamanho em bytes para gigabytes (GB)
def convert_to_gb(size_bytes):
    return size_bytes / (1024 ** 3)  # 1 GB = 1024^3 bytes


# Listar arquivos
# files = [f for f in os.listdir(raw_dataset_path) if os.path.isfile(os.path.join(raw_dataset_path, f))]

# Listar arquivos
files = [f for f in os.listdir(raw_dataset_path) if os.path.isfile(os.path.join(raw_dataset_path, f))]
file_count = len(files)


files = [f for f in os.listdir(raw_dataset_path) if f != '.DS_Store']

# Ordenar a lista
sorted_files = sorted(files, key=extract_number)

# Lista para armazenar o nome e o tamanho dos arquivos em GB
file_sizes_gb = []

# Ler o tamanho de cada arquivo em GB
for file in sorted_files:
    file_path = os.path.join(raw_dataset_path, file)
    file_size_bytes = os.path.getsize(file_path)
    file_size_gb = convert_to_gb(file_size_bytes)
    file_sizes_gb.append((file, file_size_gb))
    logging.info(f"Arquivo: {file}, Tamanho: {file_size_gb:.2f} GB")

# Calcular o tamanho total dos arquivos em GB
total_size_gb = sum(size for _, size in file_sizes_gb)
logging.info(f"Tamanho total dos arquivos: {total_size_gb:.2f} GB")

# Agrupar arquivos em novos arquivos de até 16 GB
max_size_per_file_gb = 10  # Tamanho máximo por arquivo
new_files = []  # Lista para armazenar os grupos de arquivos
current_group = []  # Grupo atual de arquivos
current_group_size = 0  # Tamanho do grupo atual em GB

for file, size_gb in file_sizes_gb:
    if current_group_size + size_gb <= max_size_per_file_gb:
        current_group.append(file)
        current_group_size += size_gb
    else:
        new_files.append((current_group, current_group_size))
        current_group = [file]
        current_group_size = size_gb

# Adicionar o último grupo, se houver
if current_group:
    new_files.append((current_group, current_group_size))

# Exibir os novos grupos de arquivos
for i, (group, group_size) in enumerate(new_files, start=1):
    logging.info(f"Novo arquivo {i}: {group}, Tamanho: {group_size:.2f} GB")

# Combinar os arquivos de cada grupo e salvar em .parquet
for i, (group, _) in enumerate(new_files, start=1):
    logging.info(client)

    # Ler todos os arquivos do grupo com Dask
    table = pq.read_table([os.path.join(raw_dataset_path, file) for file in group])
    logging.info("Partições lidas com sucesso.")

    # Caminho para salvar o novo arquivo
    output_file = os.path.join(output_path, f'combined_file_{i}.parquet')

    # Salvar o novo arquivo combinado
    pq.write_table(table, output_file)
    logging.info(f"Arquivo combinado salvo: {output_file}")
    del table
    gc.collect()

    # # Ler todos os arquivos do grupo com Dask
    # df = dd.read_parquet([os.path.join(raw_dataset_path, file) for file in group])

    # # Caminho para salvar o novo arquivo
    # output_file = os.path.join(output_path, f'combined_file_{i}.parquet')

    # # Salvar o novo arquivo combinado
    # df.to_parquet(output_file)
    # logging.info(f"Novo arquivo salvo: {output_file}")

# Fechar o cliente Dask
client.close()
logging.info("Client Dask fechado.")



INFO:root:<Client: 'tcp://127.0.0.1:53865' processes=10 threads=10, memory=59.60 GiB>
INFO:root:Arquivo: BTCUSDT-Dataset-part-1.parquet, Tamanho: 3.01 GB
INFO:root:Arquivo: BTCUSDT-Dataset-part-2.parquet, Tamanho: 3.08 GB
INFO:root:Arquivo: BTCUSDT-Dataset-part-3.parquet, Tamanho: 3.03 GB
INFO:root:Arquivo: BTCUSDT-Dataset-part-4.parquet, Tamanho: 2.90 GB
INFO:root:Arquivo: BTCUSDT-Dataset-part-5.parquet, Tamanho: 2.78 GB
INFO:root:Arquivo: BTCUSDT-Dataset-part-6.parquet, Tamanho: 2.97 GB
INFO:root:Arquivo: BTCUSDT-Dataset-part-7.parquet, Tamanho: 2.95 GB
INFO:root:Arquivo: BTCUSDT-Dataset-part-8.parquet, Tamanho: 2.91 GB
INFO:root:Arquivo: BTCUSDT-Dataset-part-9.parquet, Tamanho: 2.91 GB
INFO:root:Arquivo: BTCUSDT-Dataset-part-10.parquet, Tamanho: 2.65 GB
INFO:root:Arquivo: BTCUSDT-Dataset-part-11.parquet, Tamanho: 2.56 GB
INFO:root:Arquivo: BTCUSDT-Dataset-part-12.parquet, Tamanho: 2.63 GB
INFO:root:Arquivo: BTCUSDT-Dataset-part-13.parquet, Tamanho: 2.52 GB
INFO:root:Arquivo: BTCUSDT